In [1]:
import cProfile
import os
import random as rd
import time
import math
from collections import defaultdict
import numpy as np
import itertools as it
import Utilities.experiment_utils as eu
import unit_test as ut
import Games.mnk as mnk
import Agents.random as arand
import Agents.vanilla_mcts as mcts 
import Agents.siea_mcts as siea_mcts
import Games.function_optimisation as fo
import Utilities.logs_management as lm
#cProfile.run("wins =  random_games(10000, base_gs)")

In [ ]:

function_index = 1
runs = 3
random_seed = 1
iterations = 50
game_state = fo.GameState(function_index=function_index)
game_state.set_initial_state()
#mcts_player = siea_mcts.SIEA_MCTS_Player(max_iterations=iterations, logs=True)
mcts_player = mcts.MCTS_Player(max_iterations=iterations, logs=True)
action = eu.mcts_decision_analysis(game_state, 
                                   mcts_player, 
                                   os.path.join("Outputs","FO_single_decision_5000it","Function_" + str(function_index), mcts_player.name), 
                                   runs, 
                                   random_seed)

In [ ]:
#FOP experiment
random_seed = 1
runs = 30
iterations = 5000
es_lambda = 4
es_fitness_iterations = 30
es_generations = 20
c_list = [0.5, 1, math.sqrt(2),2,3]
c_names = ["0_5", "1", "1_4142", "2", "3"]
agents = [mcts.MCTS_Player(max_iterations=iterations, 
                           logs=True, 
                           c=c, 
                           name = "MCTS_c" + c_names[i]) for i,c in enumerate(c_list)]
agents = agents + [siea_mcts.SIEA_MCTS_Player(max_iterations=its, 
                                         es_lambda=es_lambda, 
                                         es_fitness_iterations=es_fitness_iterations,
                                        es_generations=es_generations,
                                        name = "SIEA_MCTS_its" + str(its),
                                         logs=True) for its in [iterations, iterations-(es_fitness_iterations*es_generations*es_lambda)]]
for function_index in [0,1,2,3,4]:
    game_state = fo.GameState(function_index=function_index, n_players=1)
    game_state.set_initial_state()
    for agent in agents:
        action = eu.mcts_decision_analysis(game_state = game_state,
                                             mcts_player = agent,
                                             logs_path = os.path.join("Outputs","FO_single_decision_5000it","Function_" + str(function_index), agent.name),
                                             runs = runs,
                                             random_seed = random_seed)                                        



In [2]:
#tested_agent.evolution_logs
#tested_agent.choose_action_logs
#print(tested_agent.view_mcts_tree())
game = mnk.GameState(11,11,5)
game.set_initial_state()
agent2 = mcts.MCTS_Player(max_iterations=500, rollouts = 50, logs=True)
agent1 = siea_mcts.SIEA_MCTS_Player(max_iterations=500, rollouts = 50, logs=True)
agent0 = arand.RandomPlayer()
game_player = eu.GamePlayer(game, [agent2, agent1])
game_player.play_games(50, logs=True)
#game_player.logs_by_action
game_player.save_data(os.path.join("Outputs","MNK_500it_50r", str(game_player)))

game_player = eu.GamePlayer(game, [agent1, agent2])
game_player.play_games(50, logs=True)
#game_player.logs_by_action
game_player.save_data(os.path.join("Outputs","MNK_500it_50r", str(game_player)))

In [ ]:
ut.run()